In [1]:
import torch
import io
import os
import json
from botorch.models import SingleTaskGP, FixedNoiseGP
from botorch.fit import fit_gpytorch_mll
from botorch.acquisition.multi_objective import ExpectedHypervolumeImprovement
from gpytorch.mlls import ExactMarginalLogLikelihood
import asyncio
dtype = torch.float64
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def x_to_filename(x):
    return f'{x[0]}_{x[1]}_{x[2]}_{x[3]}.json'

def filename_to_x(filename):
    return [float(x) for x in filename[:-5].split('_')]

async def evaluate_profile(x):
    filename=x_to_filename(x)
    #display(filename)
    existing_result_files=os.listdir("case/results")
    existing_results_x=[filename_to_x(filename) for filename in existing_result_files]
    distances=[torch.dist(torch.tensor(x),torch.tensor(existing_x)).item() for existing_x in existing_results_x]
    #display(distances)
    if len(distances)==0 or min(distances)>0.001:
        config=json.dumps({
            'p':x[0],
            'm':x[1],
            't':x[2],
            'a':x[3]
        })
        p=await asyncio.create_subprocess_shell(f'cd case; ./evaluate_airfoil.sh \'{config}\'',stdout=asyncio.subprocess.PIPE)
        await p.wait()
        #os.system(f'cd case; ./evaluate_airfoil.sh \'{config}\'')
    existing_result_files=os.listdir("case/results")
    existing_results_x=[filename_to_x(filename) for filename in existing_result_files]
    distances=[torch.dist(torch.tensor(x),torch.tensor(existing_x)).item() for existing_x in existing_results_x]
    min_distance_index=distances.index(min(distances))
    with open("case/results/"+existing_result_files[min_distance_index]) as file:
        result=json.load(file)
        #display(result)
        return [result['C_L'],result['C_L']/result['C_D']]


await evaluate_profile([0.42,0.05,0.18,14])

[1.46527, 10.363321309852184]

In [2]:
x_lims=[(0.1,0.9),(-0.2,0.2),(0.05,0.3),(0.01,35)]

random_initial_X=True
INITIAL_X_COUNT=20
initial_X = None
if random_initial_X or len(os.listdir("case/results"))<INITIAL_X_COUNT:
    initial_X = torch.rand(INITIAL_X_COUNT,4, dtype=dtype, device=device)
    for d in range(4):
        initial_X[:,d]=x_lims[d][0]+initial_X[:,d]*(x_lims[d][1]-x_lims[d][0])
else:
    # get initial X from filenames in case/results folder
    initial_X=torch.tensor([filename_to_x(filename) for filename in os.listdir("case/results")],dtype=dtype,device=device)
    # take six random points from the initial X
    initial_X = initial_X[torch.randperm(initial_X.shape[0])[:INITIAL_X_COUNT]]

display(initial_X)
y=await asyncio.gather(*[evaluate_profile(x.tolist()) for x in initial_X])
train_X=initial_X
train_Y = torch.tensor(y,dtype=dtype,device=device)

tensor([[ 5.5993e-01, -1.0800e-01,  7.7441e-02,  1.4749e+00],
        [ 4.0627e-01,  1.8807e-01,  1.8981e-01,  2.9996e+01],
        [ 6.1835e-01,  2.0245e-02,  1.4874e-01,  1.7709e+01],
        [ 4.7920e-01,  1.3469e-01,  1.8289e-01,  4.4725e+00],
        [ 4.2076e-01,  1.1872e-01,  1.0441e-01,  1.7589e+01],
        [ 8.9952e-01,  6.4930e-02,  1.0033e-01,  1.9243e+01],
        [ 3.2282e-01,  1.8458e-01,  2.5453e-01,  1.4915e+01],
        [ 7.3067e-01, -3.8997e-02,  2.7854e-01,  1.7882e+01],
        [ 4.8460e-01,  5.3158e-02,  9.4846e-02,  2.5006e+01],
        [ 7.7283e-01,  1.6549e-01,  1.5692e-01,  2.6990e+01],
        [ 4.5826e-01, -1.4955e-01,  1.5524e-01,  2.6218e+01],
        [ 5.8279e-01, -9.5630e-02,  2.3217e-01,  8.1742e+00],
        [ 8.1951e-01,  1.6133e-01,  1.0830e-01,  1.1918e+01],
        [ 7.7716e-01, -1.9350e-01,  2.9877e-01,  2.7218e+01],
        [ 5.4772e-01,  9.6025e-02,  1.6167e-01,  1.7874e+01],
        [ 6.7883e-01,  8.7365e-02,  1.0580e-01,  2.8663e+01],
        

/opt/OpenFOAM/OpenFOAM-10/bin/tools/RunFunctions: line 53: 141065 Floating point exception(core dumped) $APP_RUN "$@" > log.$LOG_SUFFIX 2>&1
/opt/OpenFOAM/OpenFOAM-10/bin/tools/RunFunctions: line 53: 138687 Segmentation fault      (core dumped) $APP_RUN "$@" > log.$LOG_SUFFIX 2>&1
/opt/OpenFOAM/OpenFOAM-10/bin/tools/RunFunctions: line 53: 146028 Segmentation fault      (core dumped) $APP_RUN "$@" > log.$LOG_SUFFIX 2>&1
/opt/OpenFOAM/OpenFOAM-10/bin/tools/RunFunctions: line 53: 151992 Segmentation fault      (core dumped) $APP_RUN "$@" > log.$LOG_SUFFIX 2>&1
/opt/OpenFOAM/OpenFOAM-10/bin/tools/RunFunctions: line 53: 152806 Segmentation fault      (core dumped) $APP_RUN "$@" > log.$LOG_SUFFIX 2>&1


In [3]:
bounds = torch.stack([torch.tensor(x_lims)[:,0], torch.tensor(x_lims)[:,1]])
bounds

tensor([[ 1.0000e-01, -2.0000e-01,  5.0000e-02,  1.0000e-02],
        [ 9.0000e-01,  2.0000e-01,  3.0000e-01,  3.5000e+01]])

In [4]:
import plotting
def get_fitted_model(train_X,train_Y):
    train_Yvar = torch.full_like(train_Y, 0.01)
    gp= FixedNoiseGP(train_X, train_Y, train_Yvar)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_mll(mll)
    return gp

model=get_fitted_model(train_X,train_Y)
plotting.plot_model(model,[0.4,0.05,0.12,-30],[0.4,0.05,0.12,30.0])

ModelFittingError: All attempts to fit the model have failed.

In [ ]:

from botorch.utils.multi_objective.box_decompositions.non_dominated import FastNondominatedPartitioning
from botorch.acquisition import UpperConfidenceBound
from botorch.optim import optimize_acqf
#print(train_X)
#print(train_Y)
N_BATCH=3
for iteration in range(1, N_BATCH + 1):
    gp=get_fitted_model(train_X,train_Y)
    #gp = SingleTaskGP(train_X, train_Y)
    #test_X=torch.rand(1,4)
    #print(test_X,": ",gp.posterior(train_X).mean,"+-",gp.posterior(train_X).variance.sqrt())

    #

    #acquisition_function = UpperConfidenceBound(gp, beta=0.1)
    partitioning=FastNondominatedPartitioning(ref_point=torch.tensor([0.0, 0.0]),Y=train_Y)
    acquisition_function = ExpectedHypervolumeImprovement(
        gp, ref_point=torch.tensor([0.0, 0.0]),
        partitioning=partitioning)

    candidate, acq_value = optimize_acqf(
        acquisition_function, bounds=bounds, q=1, num_restarts=1, batch_initial_conditions=train_X
    )
    print(candidate,": ",acq_value)
    predicted_Y=gp.posterior(candidate.unsqueeze(0)).mean
    prediceted_Y_var=gp.posterior(candidate.unsqueeze(0)).variance
    print(predicted_Y,"+-",prediceted_Y_var.sqrt())
    actual_value=await evaluate_profile(candidate.tolist())
    print(actual_value)
    # add new data to training data
    train_X = torch.cat([train_X, candidate.unsqueeze(0)])
    train_Y = torch.cat([train_Y, torch.tensor(actual_value,dtype=dtype,device=device).unsqueeze(0)])
    #print(train_Y)


/home/olep/.local/lib/python3.10/site-packages/botorch/acquisition/multi_objective/analytic.py:131: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ref_point = torch.tensor(


tensor([ 0.8414,  0.2000,  0.0500, 23.4391], dtype=torch.float64) :  tensor(0.6294, dtype=torch.float64)
tensor([[1.1659, 4.1387]], dtype=torch.float64, grad_fn=<TransposeBackward0>) +- tensor([[0.6823, 2.5765]], dtype=torch.float64, grad_fn=<SqrtBackward0>)
